In [ ]:
import requests
from urllib.parse import quote_plus

import pandas

In [ ]:
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, BooleanFilter, CDSView, tools

In [ ]:
output_notebook()

In [ ]:
z_min = 1.
z_max = 2.
lmass_max = 10.
url_search = 'http://z.mast.stsci.edu/api/v0.1/candels?catalog=candels_master_view&zphot.gt={}&zphot.lt={}&lmass.gt={}'.format(z_min, z_max, lmass_max)
r = requests.get(url_search)

In [ ]:
colnames = []
for col in r.json()['info']:
    colnames.append(col['name'])

json_data = r.json()['data']
df = pandas.DataFrame(json_data, columns = colnames)

In [ ]:
df

In [ ]:
print(colnames)

In [ ]:
gal = (df['Class_star'] < 0.1) & (df['PhotFlag']==0)

source = ColumnDataSource(df)
view = CDSView(source=source, filters=[BooleanFilter(gal)])

p = figure(
    toolbar_location="above",
    title='Mass-Redshift Plot',
    x_range=(0.5,2.5),
    y_range=(9.75,13.5))

p.xaxis.axis_label = r'z_{phot}'
p.yaxis.axis_label = r'log_{10}\,M_{\odot}'

hover = tools.HoverTool()
hover.tooltips=[("ID", "@IAU_Name")]
p.add_tools(hover)



p.circle('zphot', 'lmass', size=5, source=source, color='gray', hover_color="red", alpha=0.1)
p.circle('zphot', 'lmass', size=5, source=source, view=view, color='lightskyblue', hover_color="red")

show(p)

In [ ]:
df['PhotFlag'].describe()

In [ ]:
## Need to loop trough catalog to get other data!
url_search = F'http://z.mast.stsci.edu/api/v0.1/candels?catalog=candels_photometry_view&ra={df["RA"][0]}&dec={df["DEC"][0]}&radius=0.0001'
r = requests.get(url_search)
colnames = []
for col in r.json()['info']:
    colnames.append(col['name'])

json_data = r.json()['data']
df_phot = pandas.DataFrame(json_data, columns = colnames)

for index, row in df[1:].iterrows():
    if row['IAU_Name'].startswith('CANDELS_GDS'):
        url_search = F'http://z.mast.stsci.edu/api/v0.1/candels?catalog=candels_photometry_view&ra={row["RA"]}&dec={row["DEC"]}&radius=0.0001'
        r = requests.get(url_search)
        json_data = r.json()['data']
        df_tmp = pandas.DataFrame(json_data, columns = colnames)
        df_phot = df_id.append(df_tmp, ignore_index=True)

In [ ]:
df_phot